In [37]:
import gc
from glob import glob
import os
import numpy as np
import pandas as pd
import torch
import cv2
import nibabel as nib
from PIL import Image
import json
import matplotlib.pyplot as plt
import shutil


### Convert patient json to .nii.gz

Тк архив изображений nii.gz уже составлен, для него нужна разметка. Она в таком тоже формате, что и изображения, только вместо изображений бинарная маска. Если объекта на изображении нет, то маска нулевая

In [ ]:
def dict_images_annot(data_json):
    """словарь соотношения изображении и списка принадлежащих ему аннотаций """

    idx_img_annot = {}
    data_images = data_json['images']
    data_annotations = data_json['annotations']
    for i in range(len(data_images)):
        annot_idxs = []
        cut_img_idx = data_images[i]['id']

        for j in range(len(data_annotations)):
            if data_annotations[j]['image_id']==cut_img_idx:
                annot_idxs.append(data_annotations[j])
        idx_img_annot[data_json['images'][i]['file_name']] = annot_idxs

    return idx_img_annot

In [ ]:
def from_coord_to_mask(XY_all, H, W):
    """Перевод из координат в маску массива
    XY_all: координаты из разметки
    H, W: ширина и высота кадра """
    mask = np.zeros((H,W), dtype=np.int8)
    coordinates = []
    X = []
    Y = []
    for i in range(len(XY_all)):
        if i%2==0:
            X.append(np.round(XY_all[i],2))
        else:
            Y.append(np.round(XY_all[i],2))
    coordinates = list(zip(X,Y))
    #print(coordinates)
    contours = [np.array(coordinates, dtype=np.int32)]
    cv2.fillPoly(mask, contours, 1)
    return mask

так получилось, что разметка для папок 6,7 была сохранения в формате dumatura, остальные в COCO

In [52]:
#для формата разметки dumaturo
new_folder = '/content/drive/MyDrive/Medtech_dataset/dataset/short_v2'
path = '/content/drive/MyDrive/Medtech_dataset/dataset/original_datest'
H,W = 512,512
d = 128  #кол-во изображений в одном nii gz

#цикл по папкам пациентов
for i in [6,7]:
  empty = 0 #переменные для вывода статистики
  stone = 0
  dcm_files = os.listdir(os.path.join(path, str(i))) #путь до папки dicom файлов
  json_test = os.path.join(path, str(i) + '.json') #путь файла, содержащего данные о разметке этой папки
  #открытие json
  with open(json_test) as load_f:
      data_cur = json.load(load_f)
  data_all = data_cur['items']
  
  #создается пустой многомерный архив, нужной размерности
  mask_all = np.zeros((H,W,d), dtype=np.int8) 
  k = 0 #индекс слоя, на который будет запись
  for j in range(len(data_all)):
    data_one = data_all[j]
    name = data_one['id']
    annot = data_one['annotations']
    dcm_name = name +'.dcm'
    #если навазние файла из json не находится в 
    if dcm_name not in dcm_files:
      continue
    else:
      #если разметки к изображения нет, то сохраняется на этот слой пустая маска
      if annot==[]:
        mask = np.zeros((H,W), dtype=np.int8)
        mask_all[:,:,k] = mask
        empty +=1
        k+=1
      #иначе координаты переводятся в маску
      else:
        mask_one_all = np.zeros((H,W), dtype=np.int8)
        #если есть несколько объектов, то их маски объединяются на одном слое
        for g in range(len(annot)):
            XY_all = annot[g]['points']
            mask_one = from_coord_to_mask(XY_all, H, W)
            mask_one_all += mask_one
        mask_all[:,:,k] = mask_one_all
        stone +=1
        k+=1
        
  #полченный многомерный архив сохраняется в специальный формат nib и записывается
  nib_mask = nib.Nifti1Image(mask_all, np.eye(4))
  name_gz = 'patient_label_' + str(i) + '.nii.gz'
  print('{}: кол-во фото с камнем {}, без - {}'.format(name_gz, stone, empty))
  new_path =  os.path.join(new_folder, name_gz)
  nib.save(nib_mask,new_path)


patient_label_6.nii.gz: кол-во фото с камнем 19, без - 109
patient_label_7.nii.gz: кол-во фото с камнем 12, без - 116


In [ ]:
#для формата разметки coco
new_folder = '/content/drive/MyDrive/Medtech_dataset/dataset/short_v2'
path = '/content/drive/MyDrive/Medtech_dataset/dataset/original_datest'
H,W = 512,512
d =128

for i in [1,2,3,5,9,10]:
    
    #открытие json  
    json_test = os.path.join(path, str(i) + '.json')
    with open(json_test) as load_f:
        data_cur = json.load(load_f)
    images = data_cur['images']
    annot = data_cur['annotations']
    
    # специальный словарь в которм видно, есть у данного изображения разметка или нет. Если есть, выводится номера разметок
    dict_image_annot = dict_images_annot(data_cur)
    dcm_files = os.listdir(os.path.join(path, str(i))) #путь до папки dicom файлов

    mask_all = np.zeros((H,W,d), dtype=np.int8) #создается пустой многомерный архив, нужной размерности
    k = 0 #индекс слоя, на который будет запись
    empty = 0 #переменные для вывода статистики
    stone = 0
    
    for j,name in enumerate(list(dict_image_annot.keys())):
      dcm_name = name.replace('_{}.png'.format(str(i)), '.dcm') 
      #если в датасете не используются все оригинальные dicom файлы, то для этого проверяется соотвествие
      if dcm_name not in dcm_files: 
        continue
      else:
        #если разметки к изображения нет, то сохраняется на этот слой пустая маска
        if dict_image_annot[name]==[]:
            mask = np.zeros((H,W), dtype=np.int8)
            mask_all[:,:,k] = mask
            empty +=1
        else:
            #иначе координаты переводятся в маску
            data = dict_image_annot[name]
            mask_one_all = np.zeros((H,W), dtype=np.int8)
            #если есть несколько объектов, то их маски объединяются на одном слое
            for k in range(len(data)):
                XY_all = data[k]['segmentation'][0]
                mask_one = from_coord_to_mask(XY_all, H, W)
                mask_one_all += mask_one
            mask_all[:,:,k] = mask_one_all
            stone +=1
        k+=1
        
    #полченный многомерный архив сохраняется в специальный формат nib и записывается
    nib_mask = nib.Nifti1Image(mask_all, np.eye(4))
    name_gz = 'patient_label_' + str(i) + '.nii.gz'
    print('{}: кол-во фото с камнем {}, без - {}'.format(name_gz, stone, empty))
    new_path =  os.path.join(new_folder, name_gz)
    nib.save(nib_mask,new_path)


patient_label_1.nii.gz: кол-во фото с камнем 5, без - 123
patient_label_2.nii.gz: кол-во фото с камнем 7, без - 121
patient_label_3.nii.gz: кол-во фото с камнем 8, без - 120
patient_label_5.nii.gz: кол-во фото с камнем 9, без - 119
patient_label_9.nii.gz: кол-во фото с камнем 6, без - 122
patient_label_10.nii.gz: кол-во фото с камнем 5, без - 123
